<a href="https://colab.research.google.com/github/rmicallef/hd-experiments/blob/main/GPT_2uning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goals:

Finetune GPT-2 on per-environment (i.e., genre) fan fiction corpus, and use hand-written metascenes to generate candidates

Questions: How well does this work? What's the quality of the generated output?

How specifically do we need to fine-tune the corpus? Can a single generator work for multiple genres?

How do we use transformers here?

Let's see what DialoGPT looks like. I'm not sure it's going to be usefulfor anything other than NPC dialog, but it's worth seeing how well it holds up to button mashing.

In [1]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [2]:
cd transformers

/content/transformers


In [3]:
# this will install packages that require restarting the kernel, so it will end up needing to be run again after a kernel restart

!pip install .
#!pip install -r ./examples/requirements.txt
#!pip install -I pyarrow==0.17.1 numpy==1.18.0 future==0.18.1

Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1448634 sha256=f6d058037512a382f989e05687d036ad580abc8d85897f2260d9c8bb12c6a901
  Stored in directory: /tmp/pip-ephem-wheel-cache-nabr786x/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.1.0.dev0
    Uninstalling transformers-4.1.0.dev0:
      Successfully uninstalled transformers-4.1.0.dev0


In [4]:
# Hardware lottery!
!nvidia-smi

Sat Dec 12 01:13:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# DialoGPT Tests

'''
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")


# Let's chat for a few lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> ") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("God: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
'''

'\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\nimport torch\n\ntokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")\nmodel = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")\n\n\n# Let\'s chat for a few lines\nfor step in range(5):\n    # encode the new user input, add the eos_token and return a tensor in Pytorch\n    new_user_input_ids = tokenizer.encode(input(">> ") + tokenizer.eos_token, return_tensors=\'pt\')\n\n    # append the new user input tokens to the chat history\n    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids\n\n    # generated a response while limiting the total chat history to 1000 tokens, \n    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)\n\n    # pretty print last ouput tokens from bot\n    print("God: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_token

In [6]:
# No dialog for now, just generation. Less talk, more rokk 
# Generate text with untuned model
# Here, we'll use the command line version

prompt = '"The quick brown fox"'
temp = '0.8'
num_return_sequences = '3'

!python ./examples/text-generation/run_generation.py --model_type=gpt2 --model_name_or_path=gpt2 --prompt={prompt} --length=50 --temperature={temp} --num_return_sequences={num_return_sequences}

2020-12-12 00:47:11.476827: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/12/2020 00:47:16 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/12/2020 00:47:50 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=50, model_name_or_path='gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=3, p=0.9, padding_text='', prefix='', prompt='The quick brown fox', repetition_penalty=1.0, seed=42, stop_token=None, temperature=0.8, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
The quick brown foxes you see in the picture below, are actually the offspring of an eucalyptus. The eucalyptus is a fungus which grows in the plants around them and holds the bugs in place. The black and white eucaly
=== GENERATED SEQUENCE 2 ===
The quick brown fox is like this: she is clean and has a nice 

In [5]:
# Generate text with the code with a streamlined version of the huggingface example

import argparse
import logging
import numpy as np
import torch

from transformers import GPT2LMHeadModel,GPT2Tokenizer

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

#
# Functions to prepare models' input
#

def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


In [9]:
### Main() from huggingface example code with a few modifications

def remember_the_main(args_to_pass):
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_type", type=str, default='gpt2')
    parser.add_argument("--model_name_or_path", type=str, default=None, required=True)
    parser.add_argument("--prompt", type=str, default="")
    parser.add_argument("--length", type=int, default=20)
    parser.add_argument("--stop_token", type=str, default=None, help="Token at which text generation is stopped")
    parser.add_argument("--temperature", type=float, default=1.0, help="temperature of 1.0 has no effect, lower tend toward greedy sampling")
    parser.add_argument("--repetition_penalty", type=float, default=1.0, help="primarily useful for CTRL model; in that case, use 1.2")
    parser.add_argument("--k", type=int, default=0)
    parser.add_argument("--p", type=float, default=0.9)
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
    parser.add_argument("--num_return_sequences", type=int, default=1, help="The number of samples to generate.")
    args = parser.parse_args(args_to_pass)

    args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    args.n_gpu = torch.cuda.device_count()

    #logger.warning("device: %s, n_gpu: %s", args.device, args.n_gpu)

    set_seed(args)

    # Initialize the model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(args.model_name_or_path)
    model = GPT2LMHeadModel.from_pretrained(args.model_name_or_path)
    model.to(args.device)

    args.length = adjust_length_to_model(args.length, max_sequence_length=model.config.max_position_embeddings)
    #logger.info(args)

    prompt_text = args.prompt if args.prompt else input("Model prompt >>> ")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(args.device)

    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt

    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=args.length + len(encoded_prompt[0]),
        temperature=args.temperature,
        top_k=args.k,
        top_p=args.p,
        repetition_penalty=args.repetition_penalty,
        do_sample=True,
        num_return_sequences=args.num_return_sequences,
    )

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        #print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(args.stop_token) if args.stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
        #print(total_sequence)

    return generated_sequences
    
    

In [10]:
# Text generation helpers

def format_args(args_dict):
    args_list = []
    for k, v in args_dict.items():
        args_list.append('--{}'.format(k))
        args_list.append(str(v))
    return args_list

def generate_text(prompts, models, args_dict):

    for prompt in prompts:
        args_dict['prompt'] = prompt
        print('\n\nPROMPT: {}\n'.format(prompt))

        for model in models:
            args_dict['model_name_or_path'] = model
            
            generated_sequences = remember_the_main(format_args(args_dict))

            for i, sequence in enumerate(generated_sequences):
                print('\n** Model {}, seq {}:\n{}\n**'.format(model, i, sequence))

In [11]:
# Now we use the inline function instead of the command line

default_args_dict = {
    'num_return_sequences': 1,
    'length': 30,
    'temp': 1.0,
    'model_type': 'gpt2', #for now, we're just looking at GPT-2 based models
}

generate_text(['The quick brown fox'], ['gpt2'], default_args_dict)



PROMPT: The quick brown fox



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The quick brown foxes you see aren't as big, but they have a hard time making it hard to get on with the grass up ahead. Once they get over
**


In [11]:
# Let's vary the prompts a little

test_prompts = [
    'Harry opened the door and saw',
    'The New York Times reported today that',
    'The Daily Prophet reported today that',
    "Picard stepped off the shuttlecraft. He couldn't believe his eyes",
]

generate_text(test_prompts, ['gpt2'], default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Harry opened the door and saw none of the girls in the house, but had heard the voice from the edge of the window.

What was all this about, Dorner
**


PROMPT: The New York Times reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The New York Times reported today that for the past four years, the Chief Justice had overseen the Supreme Court's quest to regulate human experimentation in the United States. A New York federal court
**


PROMPT: The Daily Prophet reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
The Daily Prophet reported today that for the past four years, the Chief Rabbi had suggested that some Jews who have not received sabbat could end up starting her wedding, but this
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was in lockdown, and he had never seen anything like it before.

"Well, when you all go down, expect this
**


In [12]:
# Now we tune the model with a training corpus

!python /content/transformers/examples/language-modeling/run_clm.py \
--model_type=gpt2 \
--do_train \
--do_eval \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--save_steps 5000 \
\
--model_name_or_path=gpt2 \
--train_file=/content/data/potter-tune/potter-micro-train.txt \
--validation_file=/content/data/potter-tune/potter-micro-test.txt \
--num_train_epochs=1 \
--output_dir=/content/models/gpt2-potter-micro-1ep

2020-12-12 01:16:40.455442: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/content/transformers/examples/language-modeling/run_clm.py", line 357, in <module>
    main()
  File "/content/transformers/examples/language-modeling/run_clm.py", line 153, in main
    f"Output directory ({training_args.output_dir}) already exists and is not empty."
ValueError: Output directory (/content/models/gpt2-potter-micro-1ep) already exists and is not empty.Use --overwrite_output_dir to overcome.


In [13]:
# Generate text with a tuned model

generate_text(test_prompts, ['/content/models/gpt2-potter-micro-1ep'], default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
Harry opened the door and saw none of the customers in sight. Hermione and Sirius were staying in an unusual room and not much older than the bed Draco had her share, but this
**


PROMPT: The New York Times reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
The New York Times reported today that for the past four years, the Chief Justice had suggested that some individuals making threats are not considered "credible threats" because they're not being considered
**


PROMPT: The Daily Prophet reported today that



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
The Daily Prophet reported today that none of the 21-year-olds are being accused of some wrongdoing, but she could be forced to live with it for her entire adult life.
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station that had been destroyed, and had been completely destroyed, except it was not human. The space station was completely intact, and the unknown
**


In [14]:
# Now let's compare the models side by side.

test_prompts = [
    'Harry opened the door and saw',
    #'What have you done with my wand?',
    'The New York Times reported today that',
    'The Daily Prophet reported today that',
    #'There was trouble afoot at Hogwarts yesterday.',
    #'There was trouble afoot at Harvard yesterday.',
    #'There was trouble afoot at Kennedy High School yesterday.'
    #'Hermione was quite excited about tomorrow.',
    #'Melissa was quite excited about tomorrow.',
    "Picard stepped off the shuttlecraft. He couldn't believe his eyes",
]

models = [
          'gpt2',
          '/content/models/gpt2-potter-micro-1ep',
]

generate_text(test_prompts, models, default_args_dict)



PROMPT: Harry opened the door and saw



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Harry opened the door and saw none of the girls in the house, but had heard the voice from the edge of the window.

What was all this about, Dorner
**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
Harry opened the door and saw none of the customers in sight. Hermione and Sirius were staying in an unusual room and not much older than the bed Draco had her share, but this
**


PROMPT: The New York Times reported today that



...


** Model /content/models/gpt2-potter-micro-1ep, seq 0:
The Daily Prophet reported today that none of the 21-year-olds are being accused of some wrongdoing, but she could be forced to live with it for her entire adult life.
**


PROMPT: Picard stepped off the shuttlecraft. He couldn't believe his eyes



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model gpt2, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station was in lockdown, and he had never seen anything like it before.

"Well, when you all go down, expect this
**


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



** Model /content/models/gpt2-potter-micro-1ep, seq 0:
Picard stepped off the shuttlecraft. He couldn't believe his eyes, the station that had been destroyed, and had been completely destroyed, except it was not human. The space station was completely intact, and the unknown
**


In [13]:
# Refine the model with a little more tuning

!python /content/transformers/examples/language-modeling/run_clm.py \
--model_type=gpt2 \
--do_train \
--do_eval \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--save_steps 5000 \
\
--model_name_or_path=/content/models/gpt2-potter-micro-1ep \
--train_file=/content/data/potter-tune/potter-micro-train.txt \
--validation_file=/content/data/potter-tune/potter-micro-test.txt \
--num_train_epochs=4 \
--output_dir=/content/models/gpt2-potter-micro-5ep

2020-12-12 01:17:15.649353: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/content/transformers/examples/language-modeling/run_clm.py", line 357, in <module>
    main()
  File "/content/transformers/examples/language-modeling/run_clm.py", line 153, in main
    f"Output directory ({training_args.output_dir}) already exists and is not empty."
ValueError: Output directory (/content/models/gpt2-potter-micro-5ep) already exists and is not empty.Use --overwrite_output_dir to overcome.


In [18]:
# Try a smaller model: DistilGPT2

!python /content/transformers/examples/language-modeling/run_clm.py \
--model_type=gpt2 \
--do_train \
--do_eval \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--save_steps 5000 \
\
--model_name_or_path=distilgpt2 \
--train_file=/content/data/potter-tune/potter-micro-train.txt \
--validation_file=/content/data/potter-tune/potter-micro-test.txt \
--num_train_epochs=1 \
--output_dir=/content/models/distilgpt2-potter-micro-1ep \
--overwrite_output_dir

2020-12-12 02:05:43.438560: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/12/2020 02:05:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/12/2020 02:05:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/distilgpt2-potter-micro-1ep', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec12_02-05-45_20f9cb0d04

In [14]:
# Now gpt2 with the full data set

!python /content/transformers/examples/language-modeling/run_clm.py \
--model_type=gpt2 \
--do_train \
--do_eval \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--save_steps 5000 \
\
--model_name_or_path=gpt2 \
--train_file=/content/data/potter-tune/potter-full-train.txt \
--validation_file=/content/data/potter-tune/potter-full-test.txt \
--num_train_epochs=5 \
--output_dir=/content/models/gpt2-potter-full-1ep 


2020-12-12 01:17:36.444764: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/12/2020 01:17:38 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/12/2020 01:17:38 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/gpt2-potter-full-5ep', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec12_01-17-38_20f9cb0d0415', l

In [50]:
# Now gpt2-large with the full data set

!python /content/transformers/examples/language-modeling/run_clm.py \
--model_type=gpt2 \
--do_train \
--do_eval \
--per_device_train_batch_size=1 \
--per_device_eval_batch_size=1 \
--learning_rate 5e-5 \
--save_steps 5000 \
\
--model_name_or_path=gpt2-large \
--train_file=/content/data/potter-tune/potter-full-train-sents.txt \
--validation_file=/content/data/potter-tune/potter-full-test-sents.txt \
--num_train_epochs=10 \
--output_dir=/content/models/gpt2-large-potter-full-10ep 

2020-12-12 02:36:29.887749: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/12/2020 02:36:31 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/12/2020 02:36:31 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/models/gpt2-large-potter-full-10ep', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec12_02-36-31_20f9cb0d

In [ ]:
models = [
          'gpt2',
          '/content/models/gpt2-potter-micro-5ep',
          '/content/models/distilgpt2-potter-micro-5ep',
]

generate_text(test_prompts, models, default_args_dict)

In [49]:
# This is a little gadget to break up the text files on sentences to make sure nothing goes over the GPT2 tokenizer's max length of 1024

from nltk.tokenize import sent_tokenize

sents = []
with open('/content/data/potter-tune/potter-full-test.txt') as file:
    text = file.readlines()
    for line in text:
        sents += sent_tokenize(line)
print(len(text))
print(len(sents))

with open('/content/data/potter-tune/potter-full-test-sents.txt', 'w') as outfile:
    outfile.writelines([sent + '\n' for sent in sents])

from pprint import PrettyPrinter
pp = PrettyPrinter(width=1024)
pp.pprint(sents[:20])

max_len = 0
for sent in [sent + '\n' for sent in sents]:
    if len(sent) > max_len:
        max_len = len(sent)

print(max_len)


54696
92977
[' As per usual, Harry Potter’s day began with a visit to Diagon Alley, as it had done for the past eight years, ever since he had moved here with his godfather.',
 'Whilst some people might have appreciated the routine, Harry found it very tiresome and longed for something interesting',
 ' to happen, like a giant beanstalk growing in the middle of the town square or maybe a dragon flying over the roof of the Leaky Cauldron.',
 'But of course, that sort of thing only ever happened in the stories that he so eagerly devoured, and as Harry approached the town, he only saw the usual hustle and bustle of the market.',
 'The baker with the same old bread and rolls to sell, the butcher with his alarmingly vicious-looking knife, the town physician hobbling around on her rounds…',
 '“Good morning Mr Potter!”',
 '“Morning Mr Ollivander!” Harry greeted the elderly carpenter.',
 '“Where are you off to then, hmm?”',
 '“Flourish and Blotts.',
 'I just finished the most wonderful story!',

In [22]:
'''
Notes:
- tuning gpt2 on potter-micro (107 texts) takes about 6 minutes per epoch on a Tesla K80
- tuning gpt2 on potter-micro (107 texts) takes about 50 minutes per epoch without a GPU
- tuning gpt2 on potter-full (807 texts) takes about 50 minutes per epoch on a Tesla K80 - 1-epoch perplexity 18.88
- tuning distilgpt2 on potter-full (807 texts) takes about 39 minutes per epoch on a Tesla K80 - 1-epoch perplexity 27.00


Next steps
- address the padding warning
- try another model family?
- try a template?
'''

'\nNotes:\n- tuning gpt2 on potter-micro (107 texts) takes about 6 minutes per epoch on a Tesla K80\n- tuning gpt2 on potter-micro (107 texts) takes about 50 minutes per epoch without a GPU\n- tuning gpt2 on potter-full (807 texts) takes about 50 minutes per epoch on a Tesla K80 - 1-epoch perplexity 18.88\n- tuning distilgpt2 on potter-full (807 texts) takes about 39 minutes per epoch on a Tesla K80 - 1-epoch perplexity 27.00\n\n\nNext steps\n- address the padding warning\n- try another corpus\n- try another model family?\n- try a template?\n'